# Imports 

In [1]:
import patch_path

""
import torchcde
import torch


In [2]:
from mt_code.datasets import get_data, PersonData, P300Dataset
from mt_code.models import NeuralCde, OdeLstm, LmuLstm

In [3]:
LmuLstm(in_features = 2, hidden_size = 8, order = 8, out_feature = 1, theta = 0.5)

/home/alina/conda/anaconda3/envs/neiry/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LmuLstm(
  (rnn_cell): LMUCell(
    (input_encoder): Linear(in_features=2, out_features=8, bias=True)
    (hidden_encoder): Linear(in_features=8, out_features=8, bias=True)
    (memory_encoder): Linear(in_features=8, out_features=8, bias=True)
    (input_linear): Linear(in_features=2, out_features=8, bias=True)
    (hidden_linear): Linear(in_features=8, out_features=8, bias=True)
    (memory_linear): Linear(in_features=8, out_features=8, bias=True)
    (tanh): Tanh()
    (relu): ReLU()
  )
  (fc): Linear(in_features=8, out_features=1, bias=True)
)

In [4]:
import os
import torch
from torch import nn
import argparse
import torch.utils.data as data
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy, f1
from pytorch_lightning.callbacks import ModelCheckpoint

from mt_code.runners import Learner as IrregularSequenceLearner
from mt_code.datasets import load_dataset

## Functions

In [30]:
import torchmetrics

### old

### test

In [79]:
trainloader, testloader, in_features, num_classes, return_sequences, class_balance = load_dataset(
    'p300', timestamps = False, coeffs = False, batch_size = 128, data_dir = '../data/demons/nery_demons_dataset')

KeyboardInterrupt: 

In [13]:
trainloader, testloader, in_features, num_classes, return_sequences, class_balance = load_dataset(
    'activity', timestamps = False, coeffs = False, batch_size = 128, data_dir = '../data/person')

/home/alina/conda/anaconda3/envs/neiry/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


torch.Size([7769, 32, 1]) torch.Size([7769, 32, 7])


# Experiments

## P300

### CDE

In [54]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_f1',
    mode = 'max',
    dirpath='logs/models/demons/cde/',
    save_top_k = 3,
)

/home/alina/conda/anaconda3/envs/neiry/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Checkpoint directory logs/models/demons/cde/ exists and is not empty.
  warnings.warn(*args, **kwargs)


In [51]:
trainloader, testloader, in_features, num_classes, return_sequences, class_balance = load_dataset(
    'p300', timestamps=False, coeffs=True, batch_size=1024,  data_dir = '../data/demons/nery_demons_dataset')

/home/alina/conda/anaconda3/envs/neiry/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


train_x.shape:  torch.Size([45232, 35, 8])
train_y.shape:  torch.Size([45232])
Total number of train sequences: 45232
Total number of test  sequences: 11308
torch.Size([45232, 35, 1]) torch.Size([45232, 35, 8])


In [52]:
accuracy =  torchmetrics.Accuracy()
f1 =  torchmetrics.F1(num_classes, average = 'weighted')

/home/alina/conda/anaconda3/envs/neiry/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [53]:
cde = NeuralCde(
    8,#original num of features
    8,
    num_classes,

    return_sequences=False,
    #memory = True,
    #gated=True
    #solver_type='fixed_rk4',
)
learn = IrregularSequenceLearner(cde, lr=0.05, timestamps=True, class_weights = 1/class_balance)


In [ ]:


trainer = pl.Trainer(
    max_epochs=10,
    progress_bar_refresh_rate=1,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback]
    
)
trainer.fit(learn, trainloader, val_dataloaders = testloader)


In [ ]:


results = trainer.test(learn, testloader)
base_path = "results/{}".format("person_lcde")
os.makedirs(base_path, exist_ok=True)
with open("{}/pt_ode_lstm_{}.csv".format(base_path, 128), "a") as f:
    f.write("{:06f}\n".format(results[0]["val_acc"]))

In [57]:
best_path = "./logs/models/demons/cde/epoch=5-step=269.ckpt"
checkpoint = torch.load(best_path)
states = {}
for k_new, k_old in zip(cde.state_dict().keys(), checkpoint['state_dict'].keys()):
    states[k_new] = checkpoint['state_dict'].get(k_old)
cde.load_state_dict(state_dict = states)
    

<All keys matched successfully>

In [58]:
learn = IrregularSequenceLearner(cde, lr=0.01, timestamps=False,  class_weights = 1/class_balance)

results = trainer.test(learn, testloader)
base_path = "results/{}".format("person")
os.makedirs(base_path, exist_ok=True)
with open("{}/pt_ode_lstm_{}.csv".format(base_path, 128), "a") as f:
    f.write("{:06f}\n".format(results[0]["val_acc"]))

Testing: 100%|██████████| 12/12 [01:06<00:00,  5.55s/it]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_acc': 0.8007605075836182,
 'val_f1': 0.7121887803077698,
 'val_loss': 1.0005437135696411}
--------------------------------------------------------------------------------


### ODELSTM

In [9]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_f1',
    mode = 'max',
    dirpath='logs/models/demons/odelstm/',
    save_top_k = 3,
)

In [10]:
trainloader, testloader, in_features, num_classes, return_sequences, class_balance = load_dataset(
    'p300', timestamps=True, coeffs=False, batch_size=1024,  data_dir = '../data/demons/nery_demons_dataset')

NameError: name 'P300Dataset' is not defined

In [81]:
accuracy =  torchmetrics.Accuracy()
f1 =  torchmetrics.F1(num_classes, average = 'weighted')

/home/alina/conda/anaconda3/envs/neiry/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [82]:
olstm = OdeLstm(
    8,#original num of features
    8,
    num_classes,

    return_sequences=False,
    #memory = True,
    #gated=True
    #solver_type='fixed_rk4',
)
learn = IrregularSequenceLearner(olstm, lr=0.05, timestamps=True, class_weights = 1/class_balance)


In [83]:


trainer = pl.Trainer(
    max_epochs=10,
    progress_bar_refresh_rate=1,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback]
    
)
trainer.fit(learn, trainloader, val_dataloaders = testloader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type    | Params
----------------------------------
0 | model | OdeLstm | 738   
----------------------------------
738       Trainable params
0         Non-trainable params
738       Total params
0.003     Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

AssertionError: t must be strictly increasing or decreasing

In [ ]:


results = trainer.test(learn, testloader)
base_path = "results/{}".format("person_lcde")
os.makedirs(base_path, exist_ok=True)
with open("{}/pt_ode_lstm_{}.csv".format(base_path, 128), "a") as f:
    f.write("{:06f}\n".format(results[0]["val_acc"]))

### LMULSTM

In [5]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_f1',
    mode = 'max',
    dirpath='logs/models/demons/lmulstm/',
    save_top_k = 3,
)

/home/alina/conda/anaconda3/envs/neiry/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Checkpoint directory logs/models/demons/lmulstm/ exists and is not empty.
  warnings.warn(*args, **kwargs)


In [6]:
trainloader, testloader, in_features, num_classes, return_sequences, class_balance = load_dataset(
    'p300', timestamps=False, coeffs=False, batch_size=1024,  data_dir = '../data/demons/nery_demons_dataset')

train_x.shape:  torch.Size([45232, 35, 8])
train_y.shape:  torch.Size([45232])
Total number of train sequences: 45232
Total number of test  sequences: 11308
torch.Size([45232, 35, 1]) torch.Size([45232, 35, 8])


In [7]:
lmu = LmuLstm(
    8,#original num of features
    8,
    8,
    num_classes,
    theta = 5,
    return_sequences=False,
    #memory = True,
    #gated=True
    #solver_type='fixed_rk4',
)
learn = IrregularSequenceLearner(lmu, lr=0.05, timestamps=False, class_weights = 1/class_balance)


/home/alina/conda/anaconda3/envs/neiry/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:


trainer = pl.Trainer(
    max_epochs=10,
    progress_bar_refresh_rate=1,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback]
    
)
trainer.fit(learn, trainloader, val_dataloaders = testloader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name     | Type     | Params
--------------------------------------
0 | model    | LmuLstm  | 450   
1 | accuracy | Accuracy | 0     
2 | f1       | F1       | 0     
--------------------------------------
450       Trainable params
0         Non-trainable params
450       Total params
0.002     Total estimated model params size (MB)


Epoch 0:   0%|          | 0/57 [00:00<?, ?it/s]                       

../mt_code/models/lmu_torch.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return (AT.to(dtype=torch.float32), torch.tensor(B, dtype=torch.float32))


Epoch 0:  79%|███████▉  | 45/57 [00:01<00:00, 26.75it/s, loss=0.502, v_num=60, val_loss=1.200, val_f1=0.711, val_acc=0.792, train_acc=0.806, train_f1=0.719, train_loss=0.495]
Validating: 0it [00:00, ?it/s]
Epoch 1:  79%|███████▉  | 45/57 [00:01<00:00, 26.92it/s, loss=0.507, v_num=60, val_loss=0.866, val_f1=0.712, val_acc=0.801, train_acc=0.796, train_f1=0.705, train_loss=0.511]
Validating: 0it [00:00, ?it/s]
Epoch 2:  79%|███████▉  | 45/57 [00:01<00:00, 25.62it/s, loss=0.501, v_num=60, val_loss=0.920, val_f1=0.712, val_acc=0.801, train_acc=0.824, train_f1=0.745, train_loss=0.466]
Validating: 0it [00:00, ?it/s]
Epoch 3:  79%|███████▉  | 45/57 [00:01<00:00, 26.69it/s, loss=0.497, v_num=60, val_loss=0.957, val_f1=0.712, val_acc=0.801, train_acc=0.806, train_f1=0.719, train_loss=0.503]
Validating: 0it [00:00, ?it/s]
Epoch 4:  79%|███████▉  | 45/57 [00:01<00:00, 27.04it/s, loss=0.51, v_num=60, val_loss=0.956, val_f1=0.712, val_acc=0.801, train_acc=0.799, train_f1=0.709, train_loss=0.507] 
V

1

In [9]:


results = trainer.test(learn, testloader)
base_path = "results/{}".format("person_lcde")
os.makedirs(base_path, exist_ok=True)
with open("{}/pt_ode_lstm_{}.csv".format(base_path, 128), "a") as f:
    f.write("{:06f}\n".format(results[0]["val_acc"]))

Testing: 100%|██████████| 12/12 [00:00<00:00, 27.94it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_acc': 0.8007605075836182,
 'val_f1': 0.7121887803077698,
 'val_loss': 0.943478524684906}
--------------------------------------------------------------------------------


In [10]:
a = (2
    +3
    +4)

In [11]:
a

9

## Activity dataset

### CDE

In [ ]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_f1',
    mode = 'max',
    dirpath='logs/models/activity/cde/',
    save_top_k = 3,
)

In [ ]:
trainloader, testloader, in_features, num_classes, return_sequences, class_balance = load_dataset(
    timestamps=False, coeffs=True, batch_size=512, data_dir='../data/person')

In [ ]:
cde = NeuralCde(
    7,#original num of features
    8,
    num_classes,

    return_sequences=True,
    #memory = True,
    #gated=True
    #solver_type='fixed_rk4',
)
learn = IrregularSequenceLearner(cde, lr=0.05, timestamps=False, class_weights = 1/class_balance)


In [ ]:


trainer = pl.Trainer(
    max_epochs=10,
    progress_bar_refresh_rate=1,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback]
    
)
trainer.fit(learn, trainloader, val_dataloaders = testloader)


In [ ]:


results = trainer.test(learn, testloader)
base_path = "results/{}".format("person_lcde")
os.makedirs(base_path, exist_ok=True)
with open("{}/pt_ode_lstm_{}.csv".format(base_path, 128), "a") as f:
    f.write("{:06f}\n".format(results[0]["val_acc"]))

In [ ]:
best_path = checkpoint_callback.best_model_path
checkpoint = torch.load(best_path)
states = {}
for k_new, k_old in zip(cde.state_dict().keys(), checkpoint['state_dict'].keys()):
    states[k_new] = checkpoint['state_dict'].get(k_old)
cde.load_state_dict(state_dict = states)
    

In [ ]:
learn = IrregularSequenceLearner(cde, lr=0.01, timestamps=False,  class_weights = 1/class_balance)

results = trainer.test(learn, testloader)
base_path = "results/{}".format("person")
os.makedirs(base_path, exist_ok=True)
with open("{}/pt_ode_lstm_{}.csv".format(base_path, 128), "a") as f:
    f.write("{:06f}\n".format(results[0]["val_acc"]))

### ODELSTM 

In [ ]:
def run_exp(model_cls, logpath = 'logs/tmp'):
    
    checkpoint_callback = ModelCheckpoint(
        monitor='val_f1',
        mode = 'max',
        dirpath=logpath,
        save_top_k = 3,
    )
    
    trainloader, testloader, in_features, num_classes, return_sequences, class_balance = load_dataset()
    
    model = model_cls(
        in_features,
        128,
        num_classes,
        return_sequences=return_sequences,
        solver_type='fixed_rk4',
    )
    learn = IrregularSequenceLearner(model, lr=0.01, class_weights = 1/class_balance)

    trainer = pl.Trainer(
        max_epochs=10,
        progress_bar_refresh_rate=1,
        gradient_clip_val=1,
        callbacks=[checkpoint_callback]

    )
    trainer.fit(learn, trainloader, val_dataloaders = testloader)

    
    best_path = checkpoint_callback.best_model_path
    checkpoint = torch.load(best_path)
    states = {}
    for k_new, k_old in zip(ode_lstm.state_dict().keys(), checkpoint['state_dict'].keys()):
        states[k_new] = checkpoint['state_dict'].get(k_old)
    ode_lstm.load_state_dict(state_dict = states)


    learn = IrregularSequenceLearner(ode_lstm, lr=0.01, class_weights = 1/class_balance)

    results = trainer.test(learn, testloader)
    base_path = "results/{}".format("person")
    os.makedirs(base_path, exist_ok=True)
    with open("{}/pt_ode_lstm_{}.csv".format(base_path, 128), "a") as f:
        f.write("{:06f}\n".format(results[0]["val_acc"]))

In [ ]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_f1',
    mode = 'max',
    dirpath='logs/models/activity/odelstm/',
    save_top_k = 3,
)

In [ ]:

trainloader, testloader, in_features, num_classes, return_sequences, class_balance = load_dataset()


In [ ]:

ode_lstm = OdeLstm(
    in_features,
    128,
    num_classes,
    return_sequences=return_sequences,
    solver_type='fixed_rk4',
)
learn = IrregularSequenceLearner(ode_lstm, lr=0.01, class_weights = 1/class_balance)


In [ ]:
trainer = pl.Trainer(
    max_epochs=10,
    progress_bar_refresh_rate=1,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback]
    
)
trainer.fit(learn, trainloader, val_dataloaders = testloader)


In [ ]:

results = trainer.test(learn, testloader)
base_path = "results/{}".format("person")
os.makedirs(base_path, exist_ok=True)
with open("{}/pt_ode_lstm_{}.csv".format(base_path, 128), "a") as f:
    f.write("{:06f}\n".format(results[0]["val_acc"]))

In [ ]:
best_path = checkpoint_callback.best_model_path
checkpoint = torch.load(best_path)
states = {}
for k_new, k_old in zip(ode_lstm.state_dict().keys(), checkpoint['state_dict'].keys()):
    states[k_new] = checkpoint['state_dict'].get(k_old)
ode_lstm.load_state_dict(state_dict = states)
    

In [ ]:
learn = IrregularSequenceLearner(ode_lstm, lr=0.01, class_weights = 1/class_balance)

results = trainer.test(learn, testloader)
base_path = "results/{}".format("person")
os.makedirs(base_path, exist_ok=True)
with open("{}/pt_ode_lstm_{}.csv".format(base_path, 128), "a") as f:
    f.write("{:06f}\n".format(results[0]["val_acc"]))

### LMULSTM

In [5]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_f1',
    mode = 'max',
    dirpath='logs/models/activity/lmulstm/',
    save_top_k = 3,
)

/home/alina/conda/anaconda3/envs/neiry/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Checkpoint directory logs/models/activity/lmulstm/ exists and is not empty.
  warnings.warn(*args, **kwargs)


In [6]:
trainloader, testloader, in_features, num_classes, return_sequences, class_balance = load_dataset(
    timestamps=False)

torch.Size([7769, 32, 1]) torch.Size([7769, 32, 7])


In [23]:

lmu_lstm = LmuLstm(
    in_features,
    128,
    128,
    num_classes,
    5,
    return_sequences=return_sequences,
    memory = True,
    #gated=True
    #solver_type='fixed_rk4',
)
learn = IrregularSequenceLearner(lmu_lstm, lr=0.01, timestamps=False, class_weights = 1/class_balance)


In [24]:
def printgradnorm(self, grad_input, grad_output):

    print('grad_input size:', grad_input[0].size())
    print('grad_output size:', grad_output[0].size())
    print('grad_input norm:', grad_input[0].norm())

    


In [25]:
lmu_lstm.rnn_cell.memory_encoder.register_backward_hook(printgradnorm)

In [26]:
trainer = pl.Trainer(
    max_epochs=10,
    progress_bar_refresh_rate=1,
    gradient_clip_val=0.01,
    callbacks=[checkpoint_callback]
    
)
trainer.fit(learn, trainloader, val_dataloaders = testloader)



GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name     | Type     | Params
--------------------------------------
0 | model    | LmuLstm  | 69.0 K
1 | accuracy | Accuracy | 0     
2 | f1       | F1       | 0     
--------------------------------------
69.0 K    Trainable params
0         Non-trainable params
69.0 K    Total params
0.276     Total estimated model params size (MB)


Epoch 0:   0%|          | 0/77 [00:00<?, ?it/s]                       grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input nor

grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
Epoch 0:   4%|▍         | 3/77 [00:00<00:09,  7.95it/s, loss=nan, v_num=74, val_loss=nan.0, val_f1=0.269, val_acc=0.337, train_acc=0.413, train_f1=0.242, train_loss=nan.0]grad_input size: torch.Size([128])
grad_output size: torch.Size([

Epoch 0:   8%|▊         | 6/77 [00:00<00:08,  8.74it/s, loss=nan, v_num=74, val_loss=nan.0, val_f1=0.269, val_acc=0.337, train_acc=0.411, train_f1=0.239, train_loss=nan.0]grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: t

grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Siz

...

Epoch 0:  18%|█▊        | 14/77 [00:01<00:06,  9.01it/s, loss=nan, v_num=74, val_loss=nan.0, val_f1=0.269, val_acc=0.337, train_acc=0.459, train_f1=0.289, train_loss=nan.0]grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: 

grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: to

Epoch 0:  25%|██▍       | 19/77 [00:02<00:06,  8.97it/s, loss=nan, v_num=74, val_loss=nan.0, val_f1=0.269, val_acc=0.337, train_acc=0.408, train_f1=0.237, train_loss=nan.0]grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: 

Epoch 0:  29%|██▊       | 22/77 [00:02<00:06,  9.04it/s, loss=nan, v_num=74, val_loss=nan.0, val_f1=0.269, val_acc=0.337, train_acc=0.398, train_f1=0.227, train_loss=nan.0]grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
grad_input size: torch.Size([128])
grad_output size: torch.Size([128, 128])
grad_input norm: tensor(nan)
Epoch 0:  29%|██▊       | 22/77 [00:02<00:06,  8.70it/s, loss=nan, v_num=74, val_loss=nan.0, val_f1=0.269, val_acc=0.337, train_acc=0.398, train_f1=0.227, train_loss=nan.0]


1

In [ ]:

results = trainer.test(learn, testloader)
base_path = "results/{}".format("person_lmu")
os.makedirs(base_path, exist_ok=True)
with open("{}/pt_ode_lstm_{}.csv".format(base_path, 128), "a") as f:
    f.write("{:06f}\n".format(results[0]["val_acc"]))

In [ ]:
best_path = checkpoint_callback.best_model_path
checkpoint = torch.load(best_path)
states = {}
for k_new, k_old in zip(lmu_lstm.state_dict().keys(), checkpoint['state_dict'].keys()):
    states[k_new] = checkpoint['state_dict'].get(k_old)
lmu_lstm.load_state_dict(state_dict = states)
    

In [ ]:
learn = IrregularSequenceLearner(lmu_lstm, lr=0.01, timestamps=False, class_weights = 1/class_balance)

results = trainer.test(learn, testloader)
base_path = "results/{}".format("person")
os.makedirs(base_path, exist_ok=True)
with open("{}/pt_ode_lstm_{}.csv".format(base_path, 128), "a") as f:
    f.write("{:06f}\n".format(results[0]["val_acc"]))

## Toy data

In [ ]:
train_X, train_y = get_data()
######################
# input_channels=3 because we have both the horizontal and vertical position of a point in the spiral, 
# and time.
# hidden_channels=8 is the number of hidden channels for the evolving z_t, which we get to choose.
# output_channels=1 because we're doing binary classification.
######################
model =  LmuLstm(
    in_features = 2, hidden_size = 8, order = 8, 
    out_feature = 1, theta = 0.005, memory = False
)#ODELSTM(in_features=2, hidden_size=8, out_feature=1, return_sequences=False)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

######################
# Now we turn our dataset into a continuous path. We do this here via natural cubic spline interpolation.
# The resulting `train_coeffs` are some tensors describing the path.
# For most problems, it's advisable to save these coeffs and treat them as the dataset, 
# as this interpolation can take a long time.
######################
#train_coeffs = controldiffeq.natural_cubic_spline_coeffs(train_t, train_X)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma=0.75)
train_dataset = torch.utils.data.TensorDataset(train_X, train_y)
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32)
for epoch in range(30):
    for batch in train_dataloader:
        batch_x, batch_y = batch
        pred_y = model(batch_x[:, :, 1:])#, batch_x[:, :, 0]).squeeze(-1)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(
            pred_y, batch_y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    scheduler.step()
    print('Epoch: {}   Training loss: {}    lr: {}'.format(epoch, loss.item(), optimizer.param_groups[0]['lr']))


test_X, test_y = get_data()
pred_y = model(test_X[:, :, 1:]).squeeze(-1)
binary_prediction = (torch.sigmoid(pred_y) > 0.5).to(test_y.dtype)
prediction_matches = (binary_prediction == test_y).to(test_y.dtype)
proportion_correct = prediction_matches.sum() / test_y.size(0)
print('Test Accuracy: {}'.format(proportion_correct))


In [ ]:
train_X, train_y = get_data()
######################
# input_channels=3 because we have both the horizontal and vertical position of a point in the spiral, 
# and time.
# hidden_channels=8 is the number of hidden channels for the evolving z_t, which we get to choose.
# output_channels=1 because we're doing binary classification.
######################
model = OdeLstm(in_features=2, hidden_size=8, out_feature=1, return_sequences=False)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

######################
# Now we turn our dataset into a continuous path. We do this here via natural cubic spline interpolation.
# The resulting `train_coeffs` are some tensors describing the path.
# For most problems, it's advisable to save these coeffs and treat them as the dataset, 
# as this interpolation can take a long time.
######################
#train_coeffs = controldiffeq.natural_cubic_spline_coeffs(train_t, train_X)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma=0.75)
train_dataset = torch.utils.data.TensorDataset(train_X, train_y)
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32)
for epoch in range(30):
    for batch in train_dataloader:
        batch_x, batch_y = batch
        pred_y = model(batch_x[:, :, 1:], batch_x[:, :, 0]).squeeze(-1)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(
            pred_y, batch_y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    scheduler.step()
    print('Epoch: {}   Training loss: {}    lr: {}'.format(epoch, loss.item(), optimizer.param_groups[0]['lr']))



In [ ]:

test_X, test_y = get_data()
pred_y = model(test_X[:, :, 1:], test_X[:, :, 0]).squeeze(-1)
binary_prediction = (torch.sigmoid(pred_y) > 0.5).to(test_y.dtype)
prediction_matches = (binary_prediction == test_y).to(test_y.dtype)
proportion_correct = prediction_matches.sum() / test_y.size(0)
print('Test Accuracy: {}'.format(proportion_correct))


In [ ]:
num_epochs=30
train_X, train_y = get_data()

######################
# input_channels=3 because we have both the horizontal and vertical position of a point in the spiral, and time.
# hidden_channels=8 is the number of hidden channels for the evolving z_t, which we get to choose.
# output_channels=1 because we're doing binary classification.
######################
model =NeuralCde(input_channels=3, hidden_channels=8, output_channels=1)
optimizer = torch.optim.Adam(model.parameters())

######################
# Now we turn our dataset into a continuous path. We do this here via natural cubic spline interpolation.
# The resulting `train_coeffs` is a tensor describing the path.
# For most problems, it's probably easiest to save this tensor and treat it as the dataset.
######################
train_coeffs = torchcde.natural_cubic_coeffs(train_X)

train_dataset = torch.utils.data.TensorDataset(train_coeffs, train_y)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32)
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch_coeffs, batch_y = batch
        pred_y = model(batch_coeffs).squeeze(-1)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(pred_y, batch_y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print('Epoch: {}   Training loss: {}'.format(epoch, loss.item()))

test_X, test_y = get_data()
test_coeffs = torchcde.natural_cubic_coeffs(test_X)
pred_y = model(test_coeffs).squeeze(-1)
binary_prediction = (torch.sigmoid(pred_y) > 0.5).to(test_y.dtype)
prediction_matches = (binary_prediction == test_y).to(test_y.dtype)
proportion_correct = prediction_matches.sum() / test_y.size(0)
print('Test Accuracy: {}'.format(proportion_correct))
